# Porter Stemmer Algorithm
## Natural Language Processing
__Anish Sachdeva__

__DTU/2K16/MC/013__

The Porter Stemmer Algorithm is used very heavily and ubiquitously in the field of Natural Language Processing (NLP) It is used for the morphological analysis of words to find their stems. 

The stem of a word is the main part also referred to as the root.
E.g Happy → happi, dogs → dog, cats → cat, sunny day → sunni day etc.

We find these morphological roots for the tasks of information extraction, information retrieval and also searching data in a big corpus of information.

To impliment the Porter stemmer algorithm and the various steps it follows we first need to impliment a few utilit functions.

We need to determine whether a given letter is a vowel in or a consonant. There are 26 alphabets in the english alphabet out of which 5 are vowels; __a__, __e__, __i__, __o__ and __u__

In [3]:
vowels = ('a', 'e', 'i', 'o', 'u')
def is_vowel(letter):
    return letter in vowels;

In [4]:
# test with any alphabet in the following snippet
print(is_vowel('a'));

True


In the Porter stemmer algorith consonants are detected using a special machanism and sometime sdepending on teh context (surrounding leters) __y__ may also be  a vowel. To determing whether an alphabet at index `index` in string `word` is a consonant or not we impliment the following. We also take an integer `start` which denotes the starting index of the word in the string `word` which can represent a big corpora.

In [10]:
    def is_consonant(word, index, start):
        """:returns True if word[index] is a consonant."""
        if is_vowel(word[index]):
            return False
        if word[index] == 'y':
            if index == start:
                return True
            else:
                return not is_consonant(word, index - 1, start)
        return True

In [8]:
print(is_consonant('hello', 4, 0))

False


In [11]:
print(is_consonant('pony', 3, 0))

False


In [13]:
print(is_consonant('test', 2, 0))

True


In the Porter Stemmer Algorithm, we need to have a measure of the the number of times a Vowel and Consonent sequence occurs. if c is a consonant sequence and v a vowel sequence, and <..> indicates arbitrary presence,

           <c><v>       gives 0
           <c>vc<v>     gives 1
           <c>vcvc<v>   gives 2
           <c>vcvcvc<v> gives 3

we create a method m() which will measure this in a given string.

In [28]:
    def m(word):
        n = 0
        i = 0
        offset = len(word) - 1
        start = 0
        while True:
            if i > offset:
                return n
            if not is_consonant(word, i, start):
                break
            i += 1
        i += 1
        while True:
            while True:
                if i > offset:
                    return n
                if is_consonant(word, i, start):
                    break
                i += 1
            i += 1
            n += 1
            while True:
                if i > offset:
                    return n
                if not is_consonant(word, i, start):
                    break
                i += 1
            i += 1

In [39]:
for word in ['tree', 'oats', 'trees', 'troubles', 'private']:
    print(word, m(word))

tree 0
oats 1
trees 1
troubles 2
private 2


We need to detrmine whether a given string contains a vowel or not and here even a consonant can be treated as a vowel such as __y__ in the range (start, offset). We canwrite this function as follows.

In [48]:
def contains_vowel(word, start, offset):
    for i in range(start, offset + 1):
        if not is_consonant(word, i, 0):
            return True
    return False

In [53]:
print(contains_vowel('test', 0, 0))

False


In [54]:
print(contains_vowel('test', 0, 2))

True


We need to determine whether a word contains a double consonant in the word or or not in a given range (offset, start) 

In [64]:
def contains_double_consonant(word, offset, start):
    """:returns TRUE if the word contain a double consonant in the range [offset, start]"""
    if offset < (start + 1):
        return False
    if word[offset] != word[offset - 1]:
        return False
    return is_consonant(word, offset, 0)

In [65]:
print(contains_double_consonant('dropped', 4, 0))

True


Now, we need to define a function that can help is in identifying CVC forms suchthat we have a consonant-vowel-consonant and the consonant is not __x__, __y__ or __z__. e.g. _cav(e)_ , _lov(e)_ , _hop(e)_ , _crim(e)_ etc. and not _snow_ , _box_ , _tray_ ... etc.

In [72]:
    def is_of_form_cvc(word, start, i):
        if i < (start + 2) or not is_consonant(word, i, 0) or is_consonant(word, i - 1, 0) or not is_consonant(word, i - 2, 0):
            return False
        ch = word[i]
        if ch == 'w' or ch == 'x' or ch == 'y':
            return False
        return True

In [73]:
print(is_of_form_cvc('bat', 0, 2))

True


In [74]:
print(is_of_form_cvc('snow', 0, 3))

False


The porter stemmer algorithm follows the following steps.

## Step 1 A
The following word endings (suffixes) are transformed to other word endings.

SSES → SS

IES → I

SS → SS

S → &epsilon;

## Step 1 B
(m > 0) EED → EE

ED → &epsilon;

ING → &epsilon;

If the second or the third rule are successful then the following need to be done.

AT → ATE

BL → BLE

IZ → IZE

Both these steps are joined in a single function that can be implimented in python as below.